# L1 Logistic regression for mortality risk prediction on MIMIC-IV data

**Goal**: Predict 1-year mortality for patients following their first adult ICU admission.

**Our Model**: We will use the L1-regularized logistic regression (LASSO) model to create a strong, interpretable baseline.

# 📂 **Data Access & Setup**

This project uses the **MIMIC-IV v3.1** dataset. Due to the sensitive nature of clinical data and regulation, the raw datasets are not included in this repository.

**1. Requesting Access**
To run this pipeline, you must have a signed Data Use Agreement (DUA):

Training: Complete the [CITI Data or Specimens Researchers training](https://about.citiprogram.org/)

PhysioNet: Create an account and request access via the [MIMIC-IV PhysioNet Page](https://physionet.org/content/mimiciv/3.1/)

**2. Local Setup**
Once access is granted, download the following files from the MIMIC-IV dataset

hosp/patients.csv.gz

hosp/admissions.csv.gz

hosp/diagnoses_icd.csv.gz

hosp/procedures_icd.csv.gz

icu/icustays.csv.gz

**3. Running on Google Colab**
If using Google Colab, upload these files to your Google Drive and update the data_path variable at the top of the notebook to point to your Drive folder.

In [ ]:
!pip install lifelines

In [ ]:
## Import libraries
import os
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report,average_precision_score
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# 1. Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


data_path = '' # write your data path

if os.path.exists(data_path):
    print("✅ Path found! Loading files...")
else:
    print("❌ Path still not found. Try running the debug script below.")

patients = pd.read_csv(os.path.join(data_path,'patients.csv.gz'))
admissions = pd.read_csv(os.path.join(data_path,'admissions.csv.gz'))
icustays = pd.read_csv(os.path.join(data_path,'icustays.csv.gz'))
diagnoses = pd.read_csv(os.path.join(data_path,'diagnoses_icd.csv.gz'))
procedures = pd.read_csv(os.path.join(data_path,'procedures_icd.csv.gz'))

In [ ]:
# Convert key columns to datetime objects
# This is crucial for temporal calculations
patients['dod'] = pd.to_datetime(patients['dod']) ##date of death
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

In [ ]:
# Get Patient Demographics (Age)
# We merge admissions and patients to get age at admission
admissions = admissions.merge(patients[['subject_id', 'anchor_age', 'anchor_year', 'dod']], on='subject_id')


# Calculate age. Note: This is an approximation based on MIMIC's anchoring.
admissions['age_at_admission'] = admissions['anchor_age'] + (pd.to_datetime(admissions['admittime']).dt.year - admissions['anchor_year'])

In [ ]:
# 1. Count Total Unique Patients
num_patients = admissions['subject_id'].nunique()
print(f"Total number of patients: {num_patients}")

# 2. Calculate number of visits (admissions) per patient
# We group by subject_id and count unique hospital admission IDs
visits_per_patient = admissions.groupby('subject_id')['hadm_id'].nunique()

# 3. Calculate Mean and Standard Deviation (Std)
mean_visits = visits_per_patient.mean()
std_visits = visits_per_patient.std()

print(f"Mean number of visits per patient: {mean_visits:.2f}")
print(f"Standard deviation of visits: {std_visits:.2f}")

# 4. Plot the distribution using a Log Scale
plt.figure(figsize=(10, 6))
plt.hist(visits_per_patient, bins=50, color='skyblue', edgecolor='black', log=True)

plt.title('Distribution of Visits per Patient (Log Scale)')
plt.xlabel('Number of Visits')
plt.ylabel('Frequency (Log Count)')
plt.grid(axis='y', alpha=0.3)

plt.show()

# 2. Define the cohort
We want only adults, exclude patients who died in the hospital during their first icu visit

In [ ]:
#take only adult
adult_admissions = admissions[admissions['age_at_admission'] >= 18]

# Merge with icustays
adult_icu = icustays.merge(adult_admissions, on=['subject_id', 'hadm_id'])

# Find the *first* icu stay for each patient
adult_icu = adult_icu.sort_values(by='intime')
first_icu_stays = adult_icu.groupby('subject_id').first().reset_index()

# Exclude patients who died in the hospital
# `deathtime` in the admissions table is the time of in-hospital death
cohort = first_icu_stays[first_icu_stays['deathtime'].isnull()].copy()

In [ ]:
# The index date is the start of their very first adult ICU stay
cohort["index_date"] = cohort["intime"]

In [ ]:
print(f"Initial cohort (first adult ICU stay): {len(first_icu_stays)}")
print(f"Final cohort (after excluding in-hospital deaths): {len(cohort)}")

# 3. Define the Label
The label is True if the patient died within the 365-day prediction window after the index date.

In [ ]:
cohort["time_to_death"] = (cohort['dod'] - cohort['index_date']).dt.days
cohort["time_to_death"] = cohort["time_to_death"].fillna(np.inf)  # alive patients → very large number
cohort['label_1yr_mortality'] = (cohort["time_to_death"] <= 365) & (cohort["time_to_death"] > 0)
final_cohort = cohort[['subject_id', 'hadm_id', 'index_date', 'label_1yr_mortality']]
print(final_cohort.head())

In [ ]:
# 1. Calculate the number of deaths (positive cases)
num_positive = final_cohort['label_1yr_mortality'].sum()

# 2. Calculate the total cohort size (total patients)
total_size = len(final_cohort)

# 3. Calculate Prevalence
prevalence = num_positive / total_size

print(f"--- Cohort Prevalence Analysis ---")
print(f"Total Patients: {total_size}")
print(f"Positive Cases (1-yr Mortality): {num_positive}")
print(f"Prevalence: {prevalence:.2%}")

# Visualize the class imbalance
final_cohort['label_1yr_mortality'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Class Balance: 1-Year Mortality')
plt.xticks([0, 1], ['Survived', 'Died'], rotation=0)
plt.ylabel('Count')
plt.show()

# 4. Raw count matrix

In [ ]:
# We merge the diagnoses, procedure, prescription table with our final_cohort

cohort_diagnoses = diagnoses.merge(final_cohort[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])
cohort_diagnoses["features"] = cohort_diagnoses["icd_code"].values

cohort_procedures =procedures.merge(final_cohort[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])
cohort_procedures["features"] = cohort_procedures["icd_code"].values


# concat these tables
cohort_features = pd.concat([cohort_diagnoses[['subject_id', 'hadm_id', "features"]],
                            cohort_procedures[['subject_id', 'hadm_id', "features"]]])
cohort_features

In [ ]:
# Create the "Raw Count Matrix"
# We only care about *presence* (binary), not the count, for this baseline
# Use pivot_table to create the matrix:
# Rows = subject_id, Columns = icd_code, Values = 1 (if present)
print("Building feature matrix...")
feature_matrix = cohort_features[["subject_id", "features"]].pivot_table(
    index='subject_id',
    columns='features',
    aggfunc=lambda x: 1,  # Fill with 1 if the code exists for the patient
    fill_value=0          # Fill with 0 otherwise
)

print(f"Feature matrix shape: {feature_matrix.shape}") # (n_patients, n_unique_icd_codes)

In [ ]:
# Align Features (X) and Labels (y)
# Get the labels for the patients in our feature matrix
labels = final_cohort.set_index('subject_id').loc[feature_matrix.index]['label_1yr_mortality']

X = feature_matrix
y = labels

# 5. Model training/validation

In [ ]:
# Split Data into Train and Test Sets ---
# This is vital to prevent data leakage and evaluate the model properly
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=1,
    stratify=y  # Ensure same outcome prevalence in train/test
)

# Scikit-learn Pipeline
# We use 'liblinear' solver as it's good for L1 penalty.
lasso_model = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        penalty='l1',
        solver='liblinear',
        C=0.1,  # This is a hyperparameter you can tune
        random_state=42
    ))
])

# Train the Model
print("\nTraining the LASSO model...")
lasso_model.fit(X_train, y_train)
print("Model training complete.")


In [ ]:
# Evaluate on Test Set
y_pred_proba = lasso_model.predict_proba(X_test)[:, 1]
auroc = roc_auc_score(y_test, y_pred_proba)
auprc = average_precision_score(y_test, y_pred_proba)
print(f"Test Set AUROC: {auroc:.4f}")
print(f"Test Set AUPRC: {auprc:.4f}")

# Show a classification report
y_pred = lasso_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No Mortality', 'Mortality']))

# 6. Model interpretation

In [ ]:
# Get the trained model and feature names
model = lasso_model.named_steps['model']
features = X_train.columns

# Create a DataFrame of features and their coefficients
coefficients = pd.DataFrame({
    'feature': features,
    'coefficient': model.coef_[0]
})


In [ ]:
# 1. Extract coefficients from the model inside the pipeline
trained_model = lasso_model.named_steps['model']
feature_names = X.columns  # ICD codes

# 2. Create a DataFrame for importance
importance_df = pd.DataFrame({
    'icd_code': feature_names,
    'coefficient': trained_model.coef_[0]
})

# 3. Merge with d_icd_diagnoses for descriptions
# This assumes your d_icd_diagnoses table has 'icd_code' and 'long_title'
importance_df = importance_df.merge(
    d_icd_diagnoses[['icd_code', 'long_title']],
    on='icd_code',
    how='left'
)

# 4. Fill missing titles (in case some codes aren't in the dictionary)
importance_df['long_title'] = importance_df['long_title'].fillna("Unknown Description")

# 5. Get Top 5 Adverse (Highest positive coefficients)
top_adverse = importance_df.sort_values(by='coefficient', ascending=False).head(5)

# 6. Get Top 5 Protective (Lowest negative coefficients)
top_protective = importance_df.sort_values(by='coefficient', ascending=True).head(5)

# --- Display Results ---
print("TOP 5 ADVERSE FEATURES (Highest Risk) ---")
print(top_adverse[['icd_code', 'coefficient', 'long_title']])

print("\n TOP 5 PROTECTIVE FEATURES (Lowest Risk) ---")
print(top_protective[['icd_code', 'coefficient', 'long_title']])

# 7. Check Sparsity (How many features were zeroed out by L1?)
num_zero = (importance_df['coefficient'] == 0).sum()
print(f"\nSparsity: {num_zero} out of {len(importance_df)} features were zeroed out by the L1 penalty.")

# 7. Patient Stratification

In [ ]:
# Stratify Patients
print("\n--- Risk Stratification ---")
test_results = pd.DataFrame({
    'true_label': y_test,
    'predicted_prob': y_pred_proba
})

# Define risk groups based on predicted probability
quantile_threshold = test_results['predicted_prob'].quantile(0.75)
test_results['risk_group'] = 'Low Risk'
test_results.loc[test_results['predicted_prob'] >= quantile_threshold, 'risk_group'] = 'High Risk'

# Compare actual mortality rates between groups
stratification_summary = test_results.groupby('risk_group')['true_label'].agg(['mean', 'count'])
stratification_summary = stratification_summary.rename(columns={'mean': 'actual_mortality_rate'})

print(stratification_summary)

# 8. Bonus: survival curves

In [ ]:
# Convert 1-year mortality label to integer (0 = alive, 1 = died)
cohort['event_observed'] = cohort['label_1yr_mortality'].values.astype(int)

# Define the duration for Kaplan-Meier:
# - If patient died within 1 year, use actual time to death
# - If patient survived, censor at 365 days
cohort['duration'] = np.where(
    cohort['label_1yr_mortality'].values,
    cohort['time_to_death'].values,
    365
)

# Ensure no duration is less than 1 day (KMF cannot handle zero duration)
cohort['duration'] = cohort['duration'].clip(lower=1)

# Merge the survival info with model predictions for plotting
merge_results = test_results.join(
    cohort[["subject_id", 'event_observed', "duration"]].set_index("subject_id")
)

# Create a Kaplan-Meier fitter and plotting axis
kmf = KaplanMeierFitter()
ax = plt.subplot(111)


# High-risk group
high_risk_data = merge_results[merge_results['risk_group'] == 'High Risk']
T_high = high_risk_data['duration']          # durations for high-risk patients
E_high = high_risk_data['event_observed']    # event indicators (0/1)
kmf.fit(T_high, event_observed=E_high, label='High Risk (Model)')
kmf.plot_survival_function(ax=ax)            # plot high-risk survival curve


# Low-risk group
low_risk_data = merge_results[merge_results['risk_group'] == 'Low Risk']
T_low = low_risk_data['duration']            # durations for low-risk patients
E_low = low_risk_data['event_observed']      # event indicators (0/1)
kmf.fit(T_low, event_observed=E_low, label='Low Risk (Model)')
kmf.plot_survival_function(ax=ax)            # plot low-risk survival curve


plt.title('Survival Plot by Predicted Risk Group')
plt.xlabel('Days After ICU Discharge (Prediction Window)')
plt.ylabel('Survival Probability')
plt.xlim(0, 365)  # Limit x-axis to 1-year window
plt.ylim(0.5, 1.0)  # Zoom in on region of interest to see separation
plt.grid()
plt.show()
